In [1]:
@file:DependsOn("io.ktor:ktor-client-core:3.1.3")
@file:DependsOn("io.ktor:ktor-client-cio:3.1.3")
@file:DependsOn("io.ktor:ktor-client-content-negotiation:3.1.3")
@file:DependsOn("io.ktor:ktor-serialization-kotlinx-json:3.1.3")
@file:DependsOn("com.charleskorn.kaml:kaml-jvm:0.67.0")
@file:DependsOn("ro.jf.funds:user-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:import-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:fund-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:account-sdk:1.0.0")
@file:DependsOn("ro.jf.funds:funds-notebook-client:1.0.0")
%use dataframe

import ro.jf.funds.client.notebook.*

val client = FundsClient()
val user = client.ensureUserExists("Johann-17.2")
user

UserTO(id=e52944de-bea4-4cf9-acde-818c262d2fa6, username=Johann-17.2)

In [2]:
import ro.jf.funds.account.sdk.AccountSdk
import ro.jf.funds.fund.sdk.FundSdk
import ro.jf.funds.importer.sdk.ImportSdk

val ACCOUNTS_YAML_FILE = "../../data/provision/accounts.yaml"
val FUNDS_YAML_FILE = "../../data/provision/funds.yaml"
val IMPORT_CONFIGURATION_JSON_FILE = "../../data/provision/import-configuration.json"
val IMPORT_CONFIGURATION_YAML_FILE = "../../data/provision/import-configuration.yaml"

val accountSdk = AccountSdk()
val fundSdk = FundSdk()
val importSdk = ImportSdk()


In [3]:
import ro.jf.funds.account.api.model.*
import ro.jf.funds.commons.model.*
import kotlinx.coroutines.runBlocking
import com.charleskorn.kaml.Yaml
import kotlinx.serialization.builtins.ListSerializer

val accounts = runBlocking {
    val existingAccounts = accountSdk.listAccounts(user.id).items
    val existingAccountNames = existingAccounts.map { it.name }.toSet()
    val newAccounts = File(ACCOUNTS_YAML_FILE).readText()
        .let { Yaml.default.decodeFromString(ListSerializer(CreateAccountTO.serializer()), it) }
        .filter { it.name !in existingAccountNames }
        .map { accountSdk.createAccount(user.id, it) }
    existingAccounts + newAccounts
}
accounts.map { it.toString() }.joinToString(separator = "\n")

AccountTO(id=f3ae4349-970a-47db-af65-e9dae9923fc3, name=Cash RON, unit=Currency(value=RON))
AccountTO(id=ec9809f1-81a5-440d-b299-1210cab6ee0a, name=Cash EUR, unit=Currency(value=EUR))
AccountTO(id=d9b499e0-587a-47ec-92a8-5cf7699438c8, name=ING RON, unit=Currency(value=RON))
AccountTO(id=ef0d031f-9226-4ed9-a3c0-34440502992d, name=ING Economy, unit=Currency(value=RON))
AccountTO(id=f440b833-0965-46b7-be4b-9f831a5a0dc6, name=Food Coupons, unit=Currency(value=RON))
AccountTO(id=610599bd-8623-4a10-af1e-c3da97fb1c05, name=Revolut RON, unit=Currency(value=RON))
AccountTO(id=49320949-bad4-4374-967a-189de5010211, name=Other, unit=Currency(value=RON))
AccountTO(id=02fcf58f-46d3-4908-b100-7f6721ca6aae, name=BT RON, unit=Currency(value=RON))
AccountTO(id=46308a27-be49-44e8-9cfe-dad62d13b395, name=BT Economy RON, unit=Currency(value=RON))

In [4]:
import ro.jf.funds.fund.api.model.*
import kotlinx.coroutines.runBlocking

val funds = runBlocking {
    val existingFunds = fundSdk.listFunds(user.id).items
    val existingFundNames = existingFunds.map { it.name }.toSet()
    val newFunds = File(FUNDS_YAML_FILE).readText()
        .let { Yaml.default.decodeFromString(ListSerializer(CreateFundTO.serializer()), it) }
        .filter { it.name !in existingFundNames }
        .map { fundSdk.createFund(user.id, it) }
    existingFunds + newFunds
}
funds.map { it.toString() }.joinToString(separator = "\n")

FundTO(id=d8811e7f-c8ec-422c-8a79-64a66a000822, name=Expenses)
FundTO(id=16fe22ea-5025-4536-9703-0c3752e65fb5, name=Work Income)
FundTO(id=5db058bd-7670-4479-88ed-bcaf5c0debe3, name=Savings)
FundTO(id=fec12aa4-81f2-4bca-93fb-ee1f0562a59f, name=Gifts)

In [5]:
import ro.jf.funds.importer.api.model.*
import kotlinx.serialization.json.Json

val dataConfigurationRawYaml: String = File(IMPORT_CONFIGURATION_YAML_FILE).readText()
val importConfiguration = Yaml.default.decodeFromString(ImportConfigurationTO.serializer(), dataConfigurationRawYaml)

In [6]:
import kotlinx.coroutines.delay
import kotlinx.coroutines.runBlocking
import kotlinx.datetime.Clock
import ro.jf.funds.importer.api.model.*
import java.io.File
import java.util.*
import kotlin.time.Duration

val csvFiles =
    listOf(File("../../data/wallet/2019"), File("../../data/wallet/2020"))
        .flatMap { it.listFiles()?.toList() ?: error("no files found") }
runBlocking {
    var importTask = importSdk.import(user.id, importConfiguration, csvFiles)
    val now: Instant = Clock.System.now()
    val timeout = 60.seconds
    while (importTask.status == ImportTaskTO.Status.IN_PROGRESS && Clock.System.now() - now < timeout) {
        delay(500)
        importTask = importSdk.getImportTask(user.id, importTask.taskId)
    }
    importTask
}


ImportTaskTO(taskId=2fb58393-db26-4c1f-9e70-8237f558a3ae, status=COMPLETED, reason=null)